In [1]:
# Establish session
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/maksim.nikiforov/.ivy2/cache
The jars for the packages stored in: /Users/maksim.nikiforov/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-43beeeef-5a80-45cc-9f83-567454b66884;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 330ms :: artifacts dl 18ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default   

23/04/20 08:54:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Enable CDC globally (if you choose - you can also enable on a table-by-table basis, see https://docs.delta.io/2.0.0/delta-change-data-feed.html#enable-change-data-feed)
spark.sql("set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true")

DataFrame[key: string, value: string]

In [7]:
# Create initial table in local directory (replace path with yours)
spark.sql("CREATE TABLE changing_addresses ( \
  primaryKey int, \
  address string, \
  current boolean, \
  effectiveDate string, \
  endDate string \
  ) \
USING delta \
LOCATION '/Users/maksim.nikiforov/Documents/cdc_tests/' \
TBLPROPERTIES (delta.enableChangeDataFeed = true)" \
         )

23/04/20 08:57:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [15]:
# Populate changing_addresses Delta table with sample data
spark.sql("INSERT INTO changing_addresses \
SELECT 11 primaryKey, 'A new customer address' as address, true as current, '2023-04-20' as effectiveDate, null as endDate \
UNION \
SELECT 41 primaryKey, 'A different address' as address, true as current, '2023-04-20' as effectiveDate, null as endDate \
UNION \
SELECT 58 primaryKey, 'Yet another address' as address, true as current, '2023-04-20' as effectiveDate, null as endDate"
         )

DataFrame[]

In [37]:
# Create a temporary view with information we will merge later
spark.sql("CREATE OR REPLACE TEMPORARY VIEW updates \
AS \
SELECT 11 primaryKey, 'An updated address' as address, true as current, '2020-10-22' as effectiveDate, null as endDate \
UNION \
SELECT 99 primaryKey, 'A completely new address' as address, true as current, '2020-10-22' as effectiveDate, null as endDate" \
         )

DataFrame[]

In [20]:
# Merge the information in our view with our Delta table
spark.sql("MERGE INTO changing_addresses as original \
USING ( \
  SELECT updates.primaryKey as merge, updates.* \
  FROM updates \
  UNION ALL \
  SELECT null as merge, updates.* \
  FROM updates INNER JOIN \
  changing_addresses original on updates.primaryKey = original.primaryKey \
  WHERE original.current = true \
) mergeupdates \
ON original.primaryKey = mergeupdates.merge \
WHEN MATCHED and original.current = true THEN \
UPDATE SET current = false, endDate = mergeupdates.effectiveDate \
WHEN NOT MATCHED then \
INSERT *"
         )

23/04/20 09:02:14 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/20 09:02:15 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/20 09:02:15 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [27]:
# Create a local table in Parquet
# You do not have to enable CDC see table below for example without CDC
spark.sql("CREATE TABLE mnik_address_updates ( \
  primaryKey int, \
  address string, \
  current boolean, \
  effectiveDate string, \
  endDate string \
  ) \
USING parquet \
LOCATION '/Users/maksim.nikiforov/Documents/address_updates/' \
TBLPROPERTIES (delta.enableChangeDataFeed = true)" \
         )

DataFrame[]

In [40]:
# Example of Parquet table with no CDC
spark.sql("CREATE TABLE some_address_updates ( \
  primaryKey int, \
  address string, \
  current boolean, \
  effectiveDate string, \
  endDate string \
  ) \
USING parquet \
LOCATION '/Users/maksim.nikiforov/Documents/my_address_updates/'"
         )

DataFrame[]

In [41]:
# Populate Parquet table with some information
spark.sql("INSERT INTO some_address_updates \
SELECT 101 primaryKey, 'Yet another customer address' as address, true as current, '2018-01-02' as effectiveDate, null as endDate \
UNION \
SELECT 102 primaryKey, 'One more address' as address, true as current, '2018-01-01' as effectiveDate, null as endDate"
         )

DataFrame[]

In [33]:
# Merge local Delta table with local Parquet-only table
spark.sql("MERGE INTO changing_addresses as original \
USING ( \
  SELECT mnik_address_updates.primaryKey as merge, mnik_address_updates.* \
  FROM mnik_address_updates \
  UNION ALL \
  SELECT null as merge, mnik_address_updates.* \
  FROM mnik_address_updates INNER JOIN \
  changing_addresses original on mnik_address_updates.primaryKey = original.primaryKey \
  WHERE original.current = true \
) mergeupdates \
ON original.primaryKey = mergeupdates.merge \
WHEN MATCHED and original.current = true THEN \
UPDATE SET current = false, endDate = mergeupdates.effectiveDate \
WHEN NOT MATCHED then \
INSERT *"
         )

23/04/20 09:15:00 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/20 09:15:00 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [34]:
spark.sql("SELECT * FROM changing_addresses").show()

+----------+--------------------+-------+-------------+----------+
|primaryKey|             address|current|effectiveDate|   endDate|
+----------+--------------------+-------+-------------+----------+
|        11|  An updated address|   true|   2020-10-22|      null|
|        11|A new customer ad...|  false|   2023-04-20|2020-10-22|
|        41| A different address|   true|   2023-04-20|      null|
|        58| Yet another address|   true|   2023-04-20|      null|
|        99|A completely new ...|   true|   2020-10-22|      null|
|        22|Some customer add...|   true|    2022-1-20|      null|
|        33| A different address|   true|   2021-05-20|      null|
+----------+--------------------+-------+-------------+----------+



In [42]:
# Another merge working
spark.sql("MERGE INTO changing_addresses as original \
USING ( \
  SELECT some_address_updates.primaryKey as merge, some_address_updates.* \
  FROM some_address_updates \
  UNION ALL \
  SELECT null as merge, some_address_updates.* \
  FROM some_address_updates INNER JOIN \
  changing_addresses original on some_address_updates.primaryKey = original.primaryKey \
  WHERE original.current = true \
) mergeupdates \
ON original.primaryKey = mergeupdates.merge \
WHEN MATCHED and original.current = true THEN \
UPDATE SET current = false, endDate = mergeupdates.effectiveDate \
WHEN NOT MATCHED then \
INSERT *"
         )

23/04/20 09:21:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/20 09:21:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [43]:
spark.sql("SELECT * FROM changing_addresses").show()

+----------+--------------------+-------+-------------+----------+
|primaryKey|             address|current|effectiveDate|   endDate|
+----------+--------------------+-------+-------------+----------+
|        11|  An updated address|   true|   2020-10-22|      null|
|        11|A new customer ad...|  false|   2023-04-20|2020-10-22|
|        41| A different address|   true|   2023-04-20|      null|
|        58| Yet another address|   true|   2023-04-20|      null|
|        99|A completely new ...|   true|   2020-10-22|      null|
|       101|Yet another custo...|   true|   2018-01-02|      null|
|       102|    One more address|   true|   2018-01-01|      null|
|        22|Some customer add...|   true|    2022-1-20|      null|
|        33| A different address|   true|   2021-05-20|      null|
+----------+--------------------+-------+-------------+----------+

